In [1]:
import pandas as pd
import xlrd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 40)

In [2]:
src_path = 'Week-12/WEEK-12-DSA-STATS-WEDNESDAY.xlsx'
dest_path = 'Week-12/WEEK-12-DSA-STATS-WEDNESDAY-CLEANED.xlsx'
xls = xlrd.open_workbook(src_path, on_demand=True)
sheet_name = xls.sheet_names()[0]

writer = pd.ExcelWriter(dest_path, engine='xlsxwriter')

workbook  = writer.book
worksheet = workbook.add_worksheet(sheet_name)

header_format = workbook.add_format({'bold': True, 'font_size': 12})
header_format_normal = workbook.add_format({'font_size': 12})

list_06 = ['extra point is No Good', 'ATTEMPT FAILS', 'extra point is Blocked']
list_00 = ['punts', 'field goal is No Good', 'FUMBLES', 'INTERCEPTED', 'END GAME']
list_recovered = ['RECOVERED by', 'recovered by', 'recovers']

df = pd.read_excel(src_path, sheet_name=sheet_name)
df = df.drop_duplicates(subset=['curr_play_id'], keep='last')
df = df.reset_index(drop=True)
df = df.drop(['game_id', 'prev_play_id', 'curr_play_id', 'published_to_twitter'], axis=1)

df1 = pd.read_excel(src_path, sheet_name=sheet_name, header=None)
df1 = df1.drop_duplicates(subset=3, keep='last')
df1 = df1.reset_index(drop=True)
df1 = df1.drop([0, 1, 3, 6], axis=1)
df1 = df1.iloc[:,:27]
df1 = df1.fillna('')

# Shift down, ytg and ytg_td columns
for index in range(1, len(df1)-1):
    df1[7][index] = df1[7][index + 1]
    df1[8][index] = df1[8][index + 1]
    df1[9][index] = df1[9][index + 1]
    df['down'][index] = df['down'][index]
    df['ytg'][index] = df['ytg'][index]
    df['ytg_td'][index] = df['ytg_td'][index]

for i, row in enumerate(df1.values):
    for j, value in enumerate(row):
        worksheet.write(i, j, value, header_format_normal)

# END GAME result
worksheet.write(len(df), 6, df['home'][df.index[-2]], header_format_normal)
worksheet.write(len(df), 7, df['away'][df.index[-2]], header_format_normal)
        
# Correct Half-Time
first = None
counter = 1
quarter_1 = []
quarter_2 = []

for item in df['description']:
    if '(:' in item:
        if first == None:
            first = df.index[df['description'] == item]
            continue
        
        if df.index[df['description'] == item] < first + 10 and counter < 2:
            pass
        else:
            counter += 1
            quarter_1.append(df.index[df['description'] == item])

first = quarter_1[0]
for i in quarter_1:
    if i < first + 10:
        quarter_2.append(i)

half_time_index = quarter_2[-1]
df['curr_play_ds'][half_time_index] = 0.00
half_time_index = half_time_index.tolist()[0]
worksheet.write(half_time_index+1, 1, 0.00, header_format)

for index in range(len(df)):
    
    text = df['description'][index]

    # Extra point is Good
    if 'extra point is GOOD' in text:
        df['curr_play_ds'][index] = 7.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

    # Extra point is No Good or 2 point conversion does not succeed
    elif any(element in text for element in list_06):
        df['curr_play_ds'][index] = 6.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
    # Touchdown
    elif 'TOUCHDOWN' in text:
            worksheet.write(index+1, 1, 6.95, header_format_normal)
    
    # Safety
    elif 'SAFETY' in text:
        df['curr_play_ds'][index] = 2.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
    # RECOVERED by or recovered by or recovers
    elif any(element in text for element in list_recovered):
        if df.iloc[:,11][index] == df.iloc[:,11][index+1]:
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)
        else:
            df['curr_play_ds'][index] = 0.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

    # NULLIFIED
    elif 'NULLIFIED' in text:
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)

    elif any(element in text for element in list_00):
        df['curr_play_ds'][index] = 0.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

    # incomplete pass and down = 4
    elif 'incomplete' in text and df['down'][index] == 4:
        df['curr_play_ds'][index] = 0.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
    
    # Field goal is GOOD
    elif 'field goal is GOOD' in text:
        df['curr_play_ds'][index] = 3.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
    
    # 2 point conversion succeeds
    elif 'ATTEMPT SUCCEEDS' in text:
        df['curr_play_ds'][index] = 8.00
        worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

    else:
        if index != half_time_index:
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)
        
    # DSA
    df['dsa'][index] = df['curr_play_ds'][index] - df['prev_play_ds'][index]
    worksheet.write(index+1, 2, df['dsa'][index], header_format_normal)

worksheet.set_column(9, 9, 110)
writer.save()

In [3]:
src_path = 'Week-13/WEEK-13-DSA-STATS-SUNDAY.xlsx'
dest_path = 'Week-13/WEEK-13-DSA-STATS-SUNDAY-CLEANED.xlsx'

xls = xlrd.open_workbook(src_path, on_demand=True)
sheet_list = xls.sheet_names()
writer = pd.ExcelWriter(dest_path, engine='xlsxwriter')

workbook  = writer.book
header_format = workbook.add_format({'bold': True, 'font_size': 12})
header_format_normal = workbook.add_format({'font_size': 12})

list_06 = ['extra point is No Good', 'ATTEMPT FAILS', 'extra point is Blocked']
list_00 = ['punts', 'punt', 'field goal is No Good', 'FUMBLES', 'INTERCEPTED', 'END GAME']
list_recovered = ['RECOVERED by', 'recovered by', 'recovers']

for sheet in sheet_list:
    
    df = pd.read_excel(src_path, sheet_name=sheet)
    df = df.drop_duplicates(subset=['curr_play_id'], keep='last')
    df = df.reset_index(drop=True)
    df = df.drop(['game_id', 'prev_play_id', 'curr_play_id', 'published_to_twitter'], axis=1)
    
    df1 = pd.read_excel(src_path, sheet_name=sheet, header=None)
    df1 = df1.drop_duplicates(subset=3, keep='last')
    df1 = df1.reset_index(drop=True)
    df1 = df1.drop([0, 1, 3, 6], axis=1)
    df1 = df1.iloc[:,:27]
    df1 = df1.fillna('')
    
    # Shift down, ytg and ytg_td columns
    for index in range(1, len(df1)-1):
        df1[7][index] = df1[7][index + 1]
        df1[8][index] = df1[8][index + 1]
        df1[9][index] = df1[9][index + 1]
        df['down'][index] = df['down'][index]
        df['ytg'][index] = df['ytg'][index]
        df['ytg_td'][index] = df['ytg_td'][index]
    
    worksheet = workbook.add_worksheet(sheet)
    worksheet.set_column(9, 9, 110)

    for i, row in enumerate(df1.values):
        for j, value in enumerate(row):
            worksheet.write(i, j, value, header_format_normal)
    
    # END GAME result
    worksheet.write(len(df), 6, df['home'][df.index[-2]], header_format_normal)
    worksheet.write(len(df), 7, df['away'][df.index[-2]], header_format_normal)
    
    # Correct Half-Time
    first = None
    counter = 1
    quarter_1, quarter_2 = [], []

    for item in df['description']:
        if '(:' in item:
            if first == None:
                first = df.index[df['description'] == item]
                continue

            if df.index[df['description'] == item] < first + 10 and counter < 2:
                pass
            else:
                counter += 1
                quarter_1.append(df.index[df['description'] == item])

    first = quarter_1[0]
    for i in quarter_1:
        if i < first + 10:
            quarter_2.append(i)

    half_time_index = quarter_2[-1]
    df['curr_play_ds'][half_time_index] = 0.00
    half_time_index = half_time_index.tolist()[0]
    worksheet.write(half_time_index+1, 1, 0.00, header_format)
    
    for index in range(len(df)):
        
        text = df['description'][index]

        # Extra point is Good
        if 'extra point is GOOD' in text:
            df['curr_play_ds'][index] = 7.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # extra point is No Good or 2 point conversion does not succeed or extra point is Blocked
        elif any(element in text for element in list_06):
            df['curr_play_ds'][index] = 6.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # Touchdown
        elif 'TOUCHDOWN' in text:
            worksheet.write(index+1, 1, 6.95, header_format_normal)
        
        # Safety
        elif 'SAFETY' in text:
            df['curr_play_ds'][index] = 2.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # RECOVERED by or recovered by or recovers
        elif any(element in text for element in list_recovered):
            if df.iloc[:,11][index] == df.iloc[:,11][index+1]:
                worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)
            else:
                df['curr_play_ds'][index] = 0.00
                worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # NULLIFIED
        elif 'NULLIFIED' in text:
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)

        elif any(element in text for element in list_00):
            df['curr_play_ds'][index] = 0.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # incomplete pass and down = 4
        elif 'incomplete' in text and df['down'][index] == 4:
            df['curr_play_ds'][index] = 0.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

        # Field Goal is Good
        elif 'field goal is GOOD' in text:
            df['curr_play_ds'][index] = 3.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)
        
        # 2 point conversion succeeds
        elif 'ATTEMPT SUCCEEDS' in text:
            df['curr_play_ds'][index] = 8.00
            worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format)

        else:
            if index != half_time_index:
                worksheet.write(index+1, 1, df['curr_play_ds'][index], header_format_normal)
    
        # DSA
        df['dsa'][index] = df['curr_play_ds'][index] - df['prev_play_ds'][index]
        worksheet.write(index+1, 2, df['dsa'][index], header_format_normal)

writer.save()